In [1]:
!hadoop fs -ls /user/cloudera/movielens

Found 4 items
-rw-r--r--   1 cloudera cloudera     207997 2017-10-12 04:23 /user/cloudera/movielens/links.csv
-rw-r--r--   1 cloudera cloudera     515700 2017-10-12 04:23 /user/cloudera/movielens/movies.csv
-rw-r--r--   1 cloudera cloudera    2580392 2017-10-12 04:23 /user/cloudera/movielens/ratings.csv
-rw-r--r--   1 cloudera cloudera     199073 2017-10-12 04:23 /user/cloudera/movielens/tags.csv


In [2]:
movies = spark.read.format("csv").option("header", True).load("/user/cloudera/movielens/movies.csv")
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [3]:
type(movies)

pyspark.sql.dataframe.DataFrame

In [4]:
movies.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [10]:
movies = spark.read.format("csv").options(header = True, inferSchema = True)\
.load("/user/cloudera/movielens/movies.csv")

movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [5]:
ratings = spark.read.format("csv").options(header = True, inferSchema = True)\
.load("/user/cloudera/movielens/ratings.csv")

ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [6]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     16|   4.0|1217897793|
|     1|     24|   1.5|1217895807|
|     1|     32|   4.0|1217896246|
|     1|     47|   4.0|1217896556|
|     1|     50|   4.0|1217896523|
|     1|    110|   4.0|1217896150|
|     1|    150|   3.0|1217895940|
|     1|    161|   4.0|1217897864|
|     1|    165|   3.0|1217897135|
|     1|    204|   0.5|1217895786|
|     1|    223|   4.0|1217897795|
|     1|    256|   0.5|1217895764|
|     1|    260|   4.5|1217895864|
|     1|    261|   1.5|1217895750|
|     1|    277|   0.5|1217895772|
|     1|    296|   4.0|1217896125|
|     1|    318|   4.0|1217895860|
|     1|    349|   4.5|1217897058|
|     1|    356|   3.0|1217896231|
|     1|    377|   2.5|1217896373|
+------+-------+------+----------+
only showing top 20 rows



## Find top 10 movies based on the highest average rating. Consider only those movies that have received at least 100 ratings. 

In [7]:
from pyspark.sql.functions import *

In [14]:
(
    ratings
    .groupBy("movieId")
    .agg(avg("rating").alias("avg_rating"), count("rating").alias("rating_count"))
    .alias("t1")
    .join(movies.alias("t2"), col("t1.movieId") == col("t2.movieId"))
    .filter("rating_count > 100")
    .orderBy(desc("avg_rating"))
).show()

+-------+------------------+------------+-------+--------------------+--------------------+
|movieId|        avg_rating|rating_count|movieId|               title|              genres|
+-------+------------------+------------+-------+--------------------+--------------------+
|    318| 4.454545454545454|         308|    318|Shawshank Redempt...|         Crime|Drama|
|    858| 4.392857142857143|         210|    858|Godfather, The (1...|         Crime|Drama|
|     50| 4.328947368421052|         228|     50|Usual Suspects, T...|Crime|Mystery|Thr...|
|   1136|4.3019480519480515|         154|   1136|Monty Python and ...|Adventure|Comedy|...|
|    527| 4.296370967741935|         248|    527|Schindler's List ...|           Drama|War|
|   1193|4.2727272727272725|         143|   1193|One Flew Over the...|               Drama|
|    608|4.2711442786069655|         201|    608|        Fargo (1996)|Comedy|Crime|Dram...|
|   2571| 4.264367816091954|         261|   2571|  Matrix, The (1999)|Action|Sci

In [9]:
movies.createOrReplaceTempView("movies")

In [10]:
sql("show tables").show()

+--------+-------------------+-----------+
|database|          tableName|isTemporary|
+--------+-------------------+-----------+
| default|           employee|      false|
| default|   employee_staging|      false|
| default|             stocks|      false|
| default|    stocks_extended|      false|
| default|     stocks_parquet|      false|
| default|            weblogs|      false|
| default|weblogs_partitioned|      false|
|        |             movies|       true|
+--------+-------------------+-----------+



In [11]:
ratings.createOrReplaceTempView("ratings")

In [12]:
sql("show tables").show()

+--------+-------------------+-----------+
|database|          tableName|isTemporary|
+--------+-------------------+-----------+
| default|           employee|      false|
| default|   employee_staging|      false|
| default|             stocks|      false|
| default|    stocks_extended|      false|
| default|     stocks_parquet|      false|
| default|            weblogs|      false|
| default|weblogs_partitioned|      false|
|        |             movies|       true|
|        |            ratings|       true|
+--------+-------------------+-----------+



In [18]:
df = sql("""
select t1.movieId, t1.title, avg(t2.rating) avg_rating, count(1) rating_count
from movies t1 join ratings t2 on t1.movieId = t2.movieId 
group by t1.movieId, t1.title 
having rating_count > 100
order by avg_rating desc
""")
df.show()

+-------+--------------------+------------------+------------+
|movieId|               title|        avg_rating|rating_count|
+-------+--------------------+------------------+------------+
|    318|Shawshank Redempt...| 4.454545454545454|         308|
|    858|Godfather, The (1...| 4.392857142857143|         210|
|     50|Usual Suspects, T...| 4.328947368421052|         228|
|   1136|Monty Python and ...|4.3019480519480515|         154|
|    527|Schindler's List ...| 4.296370967741935|         248|
|   1193|One Flew Over the...|4.2727272727272725|         143|
|    608|        Fargo (1996)|4.2711442786069655|         201|
|   2571|  Matrix, The (1999)| 4.264367816091954|         261|
|   1221|Godfather: Part I...| 4.260714285714286|         140|
|   1213|   Goodfellas (1990)|4.2592592592592595|         135|
|    912|   Casablanca (1942)|             4.236|         125|
|   1196|Star Wars: Episod...| 4.228070175438597|         228|
|   1198|Raiders of the Lo...| 4.212053571428571|      

In [19]:
df.write.format("json").save("ml-agg")

In [20]:
df.rdd.getNumPartitions()

151

In [24]:
spark.sparkContext.getConf().getAll()

[('hive.metastore.warehouse.dir', 'file:/home/cloudera/spark-warehouse/'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '2g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.app.id', 'local-1507867299086'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.driver.port', '52445')]

In [25]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [26]:
df = sql("""
select t1.movieId, t1.title, avg(t2.rating) avg_rating, count(1) rating_count
from movies t1 join ratings t2 on t1.movieId = t2.movieId 
group by t1.movieId, t1.title 
having rating_count > 100
order by avg_rating desc
""")
df.rdd.getNumPartitions()

5

In [28]:
df.coalesce(1).rdd.getNumPartitions()

1

In [30]:
df.coalesce(1).write.mode("overwrite").format("json").save("ml-agg")

In [31]:
ml_agg = spark.read.format("json").load("ml-agg")
ml_agg.show()

+------------------+-------+------------+--------------------+
|        avg_rating|movieId|rating_count|               title|
+------------------+-------+------------+--------------------+
| 4.454545454545454|    318|         308|Shawshank Redempt...|
| 4.392857142857143|    858|         210|Godfather, The (1...|
| 4.328947368421052|     50|         228|Usual Suspects, T...|
|4.3019480519480515|   1136|         154|Monty Python and ...|
| 4.296370967741935|    527|         248|Schindler's List ...|
|4.2727272727272725|   1193|         143|One Flew Over the...|
|4.2711442786069655|    608|         201|        Fargo (1996)|
| 4.264367816091954|   2571|         261|  Matrix, The (1999)|
| 4.260714285714286|   1221|         140|Godfather: Part I...|
|4.2592592592592595|   1213|         135|   Goodfellas (1990)|
|             4.236|    912|         125|   Casablanca (1942)|
| 4.228070175438597|   1196|         228|Star Wars: Episod...|
| 4.212053571428571|   1198|         224|Raiders of the

In [32]:
ml_agg = spark.read.json("ml-agg")
ml_agg.show()

+------------------+-------+------------+--------------------+
|        avg_rating|movieId|rating_count|               title|
+------------------+-------+------------+--------------------+
| 4.454545454545454|    318|         308|Shawshank Redempt...|
| 4.392857142857143|    858|         210|Godfather, The (1...|
| 4.328947368421052|     50|         228|Usual Suspects, T...|
|4.3019480519480515|   1136|         154|Monty Python and ...|
| 4.296370967741935|    527|         248|Schindler's List ...|
|4.2727272727272725|   1193|         143|One Flew Over the...|
|4.2711442786069655|    608|         201|        Fargo (1996)|
| 4.264367816091954|   2571|         261|  Matrix, The (1999)|
| 4.260714285714286|   1221|         140|Godfather: Part I...|
|4.2592592592592595|   1213|         135|   Goodfellas (1990)|
|             4.236|    912|         125|   Casablanca (1942)|
| 4.228070175438597|   1196|         228|Star Wars: Episod...|
| 4.212053571428571|   1198|         224|Raiders of the

In [33]:
df.coalesce(1).write.mode("overwrite").save("ml-agg-parquet")

In [34]:
spark.read.load("ml-agg-parquet").show()

+-------+--------------------+------------------+------------+
|movieId|               title|        avg_rating|rating_count|
+-------+--------------------+------------------+------------+
|    318|Shawshank Redempt...| 4.454545454545454|         308|
|    858|Godfather, The (1...| 4.392857142857143|         210|
|     50|Usual Suspects, T...| 4.328947368421052|         228|
|   1136|Monty Python and ...|4.3019480519480515|         154|
|    527|Schindler's List ...| 4.296370967741935|         248|
|   1193|One Flew Over the...|4.2727272727272725|         143|
|    608|        Fargo (1996)|4.2711442786069655|         201|
|   2571|  Matrix, The (1999)| 4.264367816091954|         261|
|   1221|Godfather: Part I...| 4.260714285714286|         140|
|   1213|   Goodfellas (1990)|4.2592592592592595|         135|
|    912|   Casablanca (1942)|             4.236|         125|
|   1196|Star Wars: Episod...| 4.228070175438597|         228|
|   1198|Raiders of the Lo...| 4.212053571428571|      

In [36]:
sql("select * from parquet.`/user/cloudera/ml-agg-parquet`").show()

+-------+--------------------+------------------+------------+
|movieId|               title|        avg_rating|rating_count|
+-------+--------------------+------------------+------------+
|    318|Shawshank Redempt...| 4.454545454545454|         308|
|    858|Godfather, The (1...| 4.392857142857143|         210|
|     50|Usual Suspects, T...| 4.328947368421052|         228|
|   1136|Monty Python and ...|4.3019480519480515|         154|
|    527|Schindler's List ...| 4.296370967741935|         248|
|   1193|One Flew Over the...|4.2727272727272725|         143|
|    608|        Fargo (1996)|4.2711442786069655|         201|
|   2571|  Matrix, The (1999)| 4.264367816091954|         261|
|   1221|Godfather: Part I...| 4.260714285714286|         140|
|   1213|   Goodfellas (1990)|4.2592592592592595|         135|
|    912|   Casablanca (1942)|             4.236|         125|
|   1196|Star Wars: Episod...| 4.228070175438597|         228|
|   1198|Raiders of the Lo...| 4.212053571428571|      

In [37]:
sql("select * from parquet.`/user/cloudera/ml-agg-parquet`").explain()

== Physical Plan ==
*FileScan parquet [movieId#447,title#448,avg_rating#449,rating_count#450L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://quickstart.cloudera:8020/user/cloudera/ml-agg-parquet], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<movieId:string,title:string,avg_rating:double,rating_count:bigint>


In [38]:
df.explain()

== Physical Plan ==
*Sort [avg_rating#310 DESC NULLS LAST], true, 0
+- Exchange rangepartitioning(avg_rating#310 DESC NULLS LAST, 5)
   +- *Filter (rating_count#311L > 100)
      +- *HashAggregate(keys=[movieId#0, title#1], functions=[avg(rating#17), count(1)])
         +- Exchange hashpartitioning(movieId#0, title#1, 5)
            +- *HashAggregate(keys=[movieId#0, title#1], functions=[partial_avg(rating#17), partial_count(1)])
               +- *Project [movieId#0, title#1, rating#17]
                  +- *BroadcastHashJoin [cast(movieId#0 as double)], [cast(movieId#16 as double)], Inner, BuildRight
                     :- *Project [movieId#0, title#1]
                     :  +- *Filter isnotnull(movieId#0)
                     :     +- *FileScan csv [movieId#0,title#1] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://quickstart.cloudera:8020/user/cloudera/movielens/movies.csv], PartitionFilters: [], PushedFilters: [IsNotNull(movieId)], ReadSchema: struct<movieId:stri

In [39]:
df.write.saveAsTable("ml_agg")